In [1]:
import sys
import pickle

# TO CHANGE
BASEDIR = "../../"
sys.path.insert(0, BASEDIR)

In [2]:
TRIALS = 2
FIX_FILE_PATH = "./import_fix.py"
for _ in range(TRIALS):
    try:
      from src import PersonalAI, PersonalAIConfig, QAPipelineConfig, MemPipelineConfig, \
            GraphModelConfig, EmbeddingsModelConfig, EmbedderModelConfig

      from src.db_drivers import KeyValueDriverConfig, GraphDriverConfig, VectorDriverConfig
      from src.db_drivers.kv_driver import DEFAULT_INMEMORYKV_CONFIG
      from src.db_drivers.graph_driver import DEFAULT_INMEMORYGRAPH_CONFIG
      from src.db_drivers.vector_driver import VectorDBConnectionConfig

      from src.qa_pipeline.knowledge_retriever import AStarGraphSearchConfig, AStarMetricsConfig, BFSSearchConfig, MixturedGraphSearchConfig
      from src.qa_pipeline import QueryLLMParserConfig, KnowledgeComparatorConfig, KnowledgeRetrieverConfig, QALLMGeneratorConfig

      from src.memorize_pipeline import LLMExtractorConfig, LLMUpdatorConfig

      from src.utils import NodeType, Logger
    except RuntimeError as e:
        from pathlib import Path
        fix_path = Path(FIX_FILE_PATH)
        if fix_path.is_file():
            %run {fix_path} --base_dir BASEDIR
        else:
            raise e

c:\Users\nikit\anaconda3\envs\LLM\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


#### 1. Загружем датасет с триплетами, на основе которого будет построен граф знаний

In [3]:
PKL_GRAPH_PATH = 'C:/Users/nikit/temp_files/pickled_graphs/all_triplets.pkl'

with open(PKL_GRAPH_PATH, 'rb') as f:
    formated_triplets = pickle.load(f)

In [4]:
print(len(formated_triplets))

5141


#### 2. Задаём конфигурацию графа знаний

In [5]:
# Graph model configuration
GRAPH_STORAGE_CONFIG = GraphDriverConfig(db_vendor='inmemory_graph', db_config=DEFAULT_INMEMORYGRAPH_CONFIG)
GRAPH_MODEL_CONFIG = GraphModelConfig(driver_config=GRAPH_STORAGE_CONFIG)

In [ ]:
# Vector model configuration
NODES_DB_PATH = '../../data/graph_structures/vectorized_nodes/testing' # TO CHANGE
TRIPLETS_DB_PATH = '../../data/graph_structures/vectorized_triplets/testing' # TO CHANGE
NEED_TO_CLEAR = True

VECTOR_NODES_STORAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(path=NODES_DB_PATH, need_to_clear=NEED_TO_CLEAR))
VECTOR_TRIPLETS_STIRAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(path=TRIPLETS_DB_PATH, need_to_clear=NEED_TO_CLEAR))

DEVICE = 'cuda' # TO CHANGE
EMBEDDER_MODEL_PATH = '../../models/intfloat/multilingual-e5-small' # TO CHANGE
EMBEDDER_MODEL_CONFIG = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH, device=DEVICE)

VECTOR_MODEL_CONFIG = EmbeddingsModelConfig(
    nodesdb_driver_config=VECTOR_NODES_STORAGE_CONFIG,
    tripletsdb_driver_config=VECTOR_TRIPLETS_STIRAGE_CONFIG,
    embedder_config=EMBEDDER_MODEL_CONFIG)

In [ ]:
# QA-pipeline retrieve stage configuration (configuring mixture graph search/retriever)
KV_STORAGE_CONFIG = KeyValueDriverConfig(db_vendor='inmemory_kv', db_config=DEFAULT_INMEMORYKV_CONFIG)
ASTAR_RETRIEVER_CONFIG = AStarGraphSearchConfig(
    metrics_config=AStarMetricsConfig(
        h_metric_name='ip', # TO CHANGE 
        kvdriver_config=KV_STORAGE_CONFIG),
    max_depth=20, max_passed_nodes=1000, # TO CHANGE
    accepted_node_types=[NodeType.object , NodeType.hyper, NodeType.episodic]) # TO CHANGE

BFS_RETRIEVER_CONFIG = BFSSearchConfig(
    strict_filter = True, hyper_episodic_num = 15, # TO CHANGE
    chain_triplets_num = 25, other_triplets_num = 6) # TO CHANGE

RETRIEVER_NAME = 'mixture'
RETRIEVER_CONFIG = MixturedGraphSearchConfig(
    astar_config=ASTAR_RETRIEVER_CONFIG,
    bfs_config=BFS_RETRIEVER_CONFIG
)

In [ ]:
LANGUAGE = 'en' # TO CHANGE ('ru' | 'en' | 'auto')

In [ ]:
# QA-pipeline configuration
QA_PIPELINE_CONFIG = QAPipelineConfig(
    query_parser_config=QueryLLMParserConfig(lang=LANGUAGE),
    knowledge_comparator_config=KnowledgeComparatorConfig(),
    knowledge_retriever_config=KnowledgeRetrieverConfig(
        retriever_method=RETRIEVER_NAME,retriever_config=RETRIEVER_CONFIG),
    answer_generator_config=QALLMGeneratorConfig(lang=LANGUAGE))

# Memorize-pipeline configuration
MEM_PIPELINE_CONFIG = MemPipelineConfig(
    extractor_config=LLMExtractorConfig(lang=LANGUAGE),
    updator_config=LLMUpdatorConfig(lang=LANGUAGE))

PERSONALAI_CONFIG = PersonalAIConfig(
    graph_struct_config=GRAPH_MODEL_CONFIG,
    embedds_struct_config=VECTOR_MODEL_CONFIG,
    qa_pipeline_config=QA_PIPELINE_CONFIG,
    mem_pipeline_config=MEM_PIPELINE_CONFIG,
    log=Logger('log/main'))

#### 3. Инициализируем граф знаний

In [6]:
personalai = PersonalAI(config=PERSONALAI_CONFIG)

c:\Users\nikit\anaconda3\envs\LLM\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#### 4. Добавляем в граф загруженные триплеты

In [9]:
print("uploading data to graph-storage")
graph_info = personalai.kg_model.graph_struct.create_triplets(formated_triplets)

100%|██████████| 41/41 [00:17<00:00,  2.41it/s]


In [ ]:
print("uploading data to vector-storage")
vector_info = personalai.kg_model.embeddings_struct.create_triplets(formated_triplets)

#### 5. Q&A

In [10]:
examples_questions = [
    "Какой ущерб государству от потери одного гражданина?", # 8.2 mln
    "Сколько экстренных вызовов было принято в 2022 году?", # >10 mln
    "Сколько экстренных вызовов было передано в экстренные службы в 2022 году?", # 78 675
    "На чем основан успех Яндекса?", # на сильной команде  и технологиях собственной разработки
    "Перечисли 8 умных устройств умного дома, продаваемых сбером в 2023 году" # Управляющий хаб  • Сценарная кнопка  • Розетка  • Детский светильник  • Датчики открытия  • Датчики движения  • Датчики протечки  • Датчики температуры и влажности
]

In [11]:
for question in examples_questions:
    answer, info = personalai.answer_question(question)
    print(answer)
    print("=" * 35)

Средняя оценка ущерба для государства от смерти или тяжелых травм одного человека составляет 8,2 млн рублей (данные ВШЭ за 2015 год).
В 2022 году было принято и обработано более 10 миллионов экстренных вызовов, из которых 78 675 были переданы в экстренные службы.
В 2022 году было передано в экстренные службы 78 675 экстренных вызовов.
Успех Яндекса основан на сильной команде и технологиях собственной разработки. Практически все продукты и сервисы компании созданы на основе этих технологий. Виртуальный ассистент Алиса, помимо умных колонок и ТВ-станций, присутствует в браузере, картах, навигаторе, приложениях Яндекс и Дом с Алисой, а также позволяет интегрировать устройства компании и сторонних производителей в систему умного дома. Компания успешно развивает множество бизнес-моделей, включая рекламный бизнес, райдтех, электронную коммерцию, фудтех, видео- и аудиостриминг, доставку-логистику, облачные технологии и многое другое.
Умные устройства, продаваемые Сбербанком в 2023 году, включ